# Experiments 1. Non-linear Equation System: sinus

We will minimize

$$f(w_1, w_2) = \sum\limits_{i=1}^d \left(\sum\limits_{j=1}^n A_{ij} sin(x_j)+\sum\limits_{j=1}^n B_{ij} cos(x_j) - E_i\right)^2$$

for $w_1, w_2\in\mathbb{R}^n$, $x_i \in \mathbb{R}^n$, $d\leq n$ with the condition

$$XX^\top \succeq \mu I_d,$$
where $X = (x_1 \dots x_d)^\top \in \mathbb{R}^{d\times n}$

In [1]:
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import jax
import jax.numpy as jnp
import timeit
from jax.config import config

In [2]:
from methods import gradf_inexact
from methods import GradientDescent, parse_logs, AdaptiveL, StepSize, AdaptiveNoiseGD
from methods import ConstantStepSize, AdaptiveLdelta

In [3]:
matplotlib.use('Agg')
params = {'legend.fontsize': 20,
          'legend.handlelength': 4,
          "axes.labelsize": 45,
          "xtick.labelsize": 25,
          "ytick.labelsize": 25,
          "lines.linewidth": 2,
           "axes.titlesize":30}
matplotlib.rcParams.update(params)

In [4]:
config.update("jax_enable_x64", True)

In [5]:
path_pics = "../pics/"

In [6]:
def f1(x, A, B, E):
    z = A @ jnp.sin(x) + B @ jnp.cos(x) - E
    return jnp.square(z).sum()

gradf = jax.grad(f1, argnums=0, has_aux=False)
jit_gradf = jax.jit(gradf)

## 0. Dataset

In [7]:
d = 40
n = 100
A = np.random.randn(n, n)
Q, _ = np.linalg.qr(A)
eps = 0.01
A = np.random.randn(d, d) @ Q[:d]
B = np.random.randn(d, d) @ Q[d:2*d]
xsol = np.random.randn(n)
E = A @ np.sin(xsol) + B @ np.cos(xsol)
print(np.linalg.norm(A @ B.T), np.linalg.norm(B @ A.T))
eigA = np.linalg.eig(A@A.T)[0]
eigB = np.linalg.eig(B@B.T)[0]
print(eigA.max(), eigB.max())
mu = min(min(eigA),min(eigB))
L = 8 * np.linalg.svd(np.hstack([A, B]))[1].max()**2
mu, L, L/mu

1.3022956571079577e-13 1.3022956571079577e-13
144.2655771313923 143.2602718581258


(0.0022441249060558646, 1735.6913391535613, 773437.9376431882)

In [8]:
def params(A, B, E):
    gradf = jax.grad(f1, argnums=0, has_aux=False)
    jit_gradf = jax.jit(gradf)
    gradf = lambda x: np.array(jit_gradf(x, A, B, E).block_until_ready())
    f = lambda x: f1(x, A, B, E)
    return f, gradf
f, gradf = params(A, B, E)

In [9]:
f(xsol).item(), f(np.zeros(n)).item(), gradf(xsol).shape

(0.0, 1167.8044353061005, (100,))

## 1. Noise Distributed on the Unit Sphere

The case when $\xi \sim \mathcal{U}(S_1(0))$

In [10]:
np.linalg.svd(np.hstack([A, B]))[1].max()**2

216.96141739419517

In [11]:
Delta_list = [1e-7, 1e-4, 1e-1]
Delta_list = []

v = np.random.randn(n)
d_list = [8, 32, 128]
n = 256
np.random.seed(1)
for d in d_list:
    Q, _ = np.linalg.qr(np.random.randn(n, n))
    A = np.random.rand(d, d) @ Q[:d]
    B = np.random.rand(d, d) @ Q[d:2*d]
    xsol = np.random.randn(n)
    E = A @ np.sin(xsol) + B @ np.cos(xsol)
    eigA = np.linalg.eig(A@A.T)[0]
    eigB = np.linalg.eig(B@B.T)[0]
    mu = min(min(eigA),min(eigB))
    L = 8 * np.sqrt(2) * np.linalg.svd(np.hstack([A, B]))[1].max()**2
    print("{}\t{}".format(d, L/mu))

8	213823.36290220183
32	51279411.44256377
128	1347207780.9634402


In [12]:
Delta_list = [1e-4, 1e-1]

v = np.random.randn(n)
d_list = [8, 32, 128]
n = 256

res = {d:{"delta":[], 
           "iters_adaptL":[], "time_adaptL":[], "adaptL,x0-x*": [], "normg_adaptL": [],
           "iters_exact":[], "time_exact":[], "exact,x0-x*": [], "normg_exact": [],
          "iters_adaptLdelta":[], "time_adaptLdelta":[], "adaptLdelta,x0-x*": [], "normg_adaptLdelta": [],
         "additer,x0-x*":[], "normg_additer":[]} for d in d_list}
mu_list = {}
number = 1
save_iter = 1
N = int(1.2e6)
methods = []
np.random.seed(1)
for d in d_list:
    Q, _ = np.linalg.qr(np.random.randn(n, n))
    A = np.random.rand(d, d) @ Q[:d]
    B = np.random.rand(d, d) @ Q[d:2*d]
    xsol = np.random.randn(n)
    E = A @ np.sin(xsol) + B @ np.cos(xsol)
    eigA = np.linalg.eig(A@A.T)[0]
    eigB = np.linalg.eig(B@B.T)[0]
    mu = min(min(eigA),min(eigB))
    L = 8 * np.sqrt(2) * np.linalg.svd(np.hstack([A, B]))[1].max()**2
    
    v = np.random.randn(n)
    f, gradf = params(A, B, E)
    print(d, L/mu)
    mu_list[d] = L/mu
    alpha = 1/L
    w = np.ones(n)
    for Delta in Delta_list:
        res[d]["delta"].append(int(np.log10(Delta)))
        tol = np.sqrt(6) * Delta

        grad_inexact = lambda w: gradf_inexact(w, gradf, Delta, 1, v=v)
        method = GradientDescent(ConstantStepSize(alpha), name="GD, Delta={}".format(Delta), save_iter=save_iter)
        x = method.solve(w, f, grad_inexact, tol=tol, max_iter=N)
        g = lambda: GradientDescent(ConstantStepSize(alpha),
                                    return_history=False).solve(w, f, grad_inexact, tol=tol, max_iter=N)
        T = timeit.timeit(g, number=number)/number
        print("\t{}\t{}\t{:.2f}\t{:.6f}\t{:.2f}\t{:.2f}".format(Delta, len(method.history), T*1000, np.linalg.norm(x-w), 
                                                np.linalg.norm(gradf(x))/Delta, f(x)))
        methods.append(method)
        res[d]["iters_exact"].append(len(method.history))
        res[d]["time_exact"].append("{:.2f}".format(T*1000))
        res[d]["exact,x0-x*"].append("{:.1f}".format(np.linalg.norm(x-w)))
        res[d]["normg_exact"].append("{:.2f}".format(np.linalg.norm(gradf(x))/Delta))


        method = GradientDescent(ConstantStepSize(alpha), name="GD, Delta={}".format(Delta), save_iter=save_iter)
        x = method.solve(w, f, grad_inexact, tol=0, max_iter=N)
        print("\t{}\t{}\t{}\t{:.6f}\t{:.2f}\t{:.2f}".format(Delta, len(method.history), "-", np.linalg.norm(x-w), 
                                                np.linalg.norm(gradf(x))/Delta, f(x)))
        methods.append(method)
        res[d]["additer,x0-x*"].append("{:.1f}".format(np.linalg.norm(x-w)))
        res[d]["normg_additer"].append("{:.2f}".format(np.linalg.norm(gradf(x))/Delta))
        

        method = GradientDescent(AdaptiveL(L0=1, Delta=Delta, Lmin=mu/4), name="GD, Delta={}".format(Delta), save_iter=save_iter)
        x = method.solve(w, f, grad_inexact, tol=tol, max_iter=N)
        g = lambda: GradientDescent(AdaptiveL(L0=1, Delta=Delta, Lmin=mu/4),
                                    return_history=False).solve(w, f, grad_inexact, tol=tol, max_iter=N)
        T = timeit.timeit(g, number=number)/number        
        print("\t{}\t{}\t{:.2f}\t{:.6f}\t{:.2f}\t{:.2f}".format(Delta, len(method.history), T*1000, np.linalg.norm(x-w), 
                                                np.linalg.norm(gradf(x))/Delta, f(x)))
        methods.append(method)
        res[d]["iters_adaptL"].append(len(method.history))
        res[d]["time_adaptL"].append("{:.2f}".format(T*1000))
        res[d]["adaptL,x0-x*"].append("{:.1f}".format(np.linalg.norm(x-w)))
        res[d]["normg_adaptL"].append("{:.2f}".format(np.linalg.norm(gradf(x))/Delta))


        method = AdaptiveNoiseGD(AdaptiveLdelta(L0=1, mindelta=1e-12, Lmin=mu/4, mu=mu), name="GD, Delta={}".format(Delta), save_iter=save_iter, alpha=np.sqrt(6))
        x = method.solve(w, f, grad_inexact, max_iter=N)
        g = lambda: AdaptiveNoiseGD(AdaptiveLdelta(L0=1, mindelta=1e-12, Lmin=mu/4, mu=mu), return_history=False, 
                                    alpha=np.sqrt(6)).solve(w, f, grad_inexact, max_iter=N)
        T = timeit.timeit(g, number=number)/number        
        print("\t{}\t{}\t{:.2f}\t{:.6f}\t{:.2f}\t{}".format(Delta, len(method.history), T*1000, np.linalg.norm(x-w), 
                                                np.linalg.norm(gradf(x))/Delta, f(x)))
        methods.append(method)
        res[d]["iters_adaptLdelta"].append(len(method.history))
        res[d]["time_adaptLdelta"].append("{:.2f}".format(T*1000))
        res[d]["adaptLdelta,x0-x*"].append("{:.1f}".format(np.linalg.norm(x-w)))
        res[d]["normg_adaptLdelta"].append("{:.2f}".format(np.linalg.norm(gradf(x))/Delta))    
        print("\n")

8 213823.36290220183
	0.0001	14434	1995.70	5.061611	2.41	0.00


KeyboardInterrupt: 

In [231]:
s = ""

for d in d_list:
    s += str(d) + " & "
    kappa = mu_list[d]
    l = int(np.log10(kappa))
    s += "${:.1f}\\cdot 10^{{{}}}$".format(kappa/10**l, l) + " & "
    
    cur_list = ["$10^{{{}}}$".format(i) for i in res[d]["delta"]]
    s+= "\\begin{tabular}{@{}c@{}} " + " \\\\ ".join(cur_list) + " \\end{tabular}&"

    cur_list = ["${}$".format(i) for i in res[d]["iters_exact"]]
    s+= "\\begin{tabular}{@{}c@{}} " + " \\\\ ".join(cur_list) + " \\end{tabular}&"
    cur_list = ["${}$".format(i) for i in res[d]["time_exact"]]
    s+= "\\begin{tabular}{@{}c@{}} " + " \\\\ ".join(cur_list) + " \\end{tabular}&"

    cur_list = ["${}$".format(i) for i in res[d]["iters_adaptL"]]
    s+= "\\begin{tabular}{@{}c@{}} " + " \\\\ ".join(cur_list) + " \\end{tabular}&"
    cur_list = ["${}$".format(i) for i in res[d]["time_adaptL"]]
    s+= "\\begin{tabular}{@{}c@{}} " + " \\\\ ".join(cur_list) + " \\end{tabular}&"
    
    cur_list = ["${}$".format(i) for i in res[d]["iters_adaptLdelta"]]
    s+= "\\begin{tabular}{@{}c@{}} " + " \\\\ ".join(cur_list) + " \\end{tabular}&"
    cur_list = ["${}$".format(i) for i in res[d]["time_adaptLdelta"]]
    s+= "\\begin{tabular}{@{}c@{}} " + " \\\\ ".join(cur_list) + " \\end{tabular}"

    s+= "\\\\\n\\hline\n"
print(s)

8 & $2.1\cdot 10^{5}$ & \begin{tabular}{@{}c@{}} $10^{-4}$ \\ $10^{-1}$ \end{tabular}&\begin{tabular}{@{}c@{}} $14434$ \\ $1479$ \end{tabular}&\begin{tabular}{@{}c@{}} $2203.85$ \\ $220.82$ \end{tabular}&\begin{tabular}{@{}c@{}} $508$ \\ $56$ \end{tabular}&\begin{tabular}{@{}c@{}} $447.06$ \\ $45.54$ \end{tabular}&\begin{tabular}{@{}c@{}} $306$ \\ $69$ \end{tabular}&\begin{tabular}{@{}c@{}} $813.99$ \\ $178.63$ \end{tabular}\\
\hline
32 & $5.0\cdot 10^{6}$ & \begin{tabular}{@{}c@{}} $10^{-4}$ \\ $10^{-1}$ \end{tabular}&\begin{tabular}{@{}c@{}} $59643$ \\ $12536$ \end{tabular}&\begin{tabular}{@{}c@{}} $10314.68$ \\ $2290.65$ \end{tabular}&\begin{tabular}{@{}c@{}} $1871$ \\ $383$ \end{tabular}&\begin{tabular}{@{}c@{}} $1808.70$ \\ $393.74$ \end{tabular}&\begin{tabular}{@{}c@{}} $510$ \\ $144$ \end{tabular}&\begin{tabular}{@{}c@{}} $1509.95$ \\ $499.39$ \end{tabular}\\
\hline
128 & $7.7\cdot 10^{8}$ & \begin{tabular}{@{}c@{}} $10^{-4}$ \\ $10^{-1}$ \end{tabular}&\begin{tabular}{@{}c@{}} $

In [232]:
s = ""

for d in d_list:
    s += str(d) + " & "
    kappa = mu_list[d]
    l = int(np.log10(kappa))
    s += "${:.1f}\\cdot 10^{{{}}}$".format(kappa/10**l, l) + " & "
    cur_list = ["$10^{{{}}}$".format(i) for i in res[d]["delta"]]
    
    s+= "\\begin{tabular}{@{}c@{}} " + " \\\\ ".join(cur_list) + " \\end{tabular}&"
    cur_list = ["${}$".format(i) for i in res[d]["exact,x0-x*"]]
    s+= "\\begin{tabular}{@{}c@{}} " + " \\\\ ".join(cur_list) + " \\end{tabular}&"
    cur_list = ["${}$".format(i) for i in res[d]["normg_exact"]]
    s+= "\\begin{tabular}{@{}c@{}} " + " \\\\ ".join(cur_list) + " \\end{tabular}&"


    cur_list = ["${}$".format(i) for i in res[d]["adaptL,x0-x*"]]
    s+= "\\begin{tabular}{@{}c@{}} " + " \\\\ ".join(cur_list) + " \\end{tabular}&"
    cur_list = ["${}$".format(i) for i in res[d]["normg_adaptL"]]
    s+= "\\begin{tabular}{@{}c@{}} " + " \\\\ ".join(cur_list) + " \\end{tabular}&"
    
    cur_list = ["${}$".format(i) for i in res[d]["adaptLdelta,x0-x*"]]
    s+= "\\begin{tabular}{@{}c@{}} " + " \\\\ ".join(cur_list) + " \\end{tabular}&"
    cur_list = ["${}$".format(i) for i in res[d]["normg_adaptLdelta"]]
    s+= "\\begin{tabular}{@{}c@{}} " + " \\\\ ".join(cur_list) + " \\end{tabular}"

    s+= "\\\\\n\\hline\n"
print(s)

8 & $2.1\cdot 10^{5}$ & \begin{tabular}{@{}c@{}} $10^{-4}$ \\ $10^{-1}$ \end{tabular}&\begin{tabular}{@{}c@{}} $5.1$ \\ $4.8$ \end{tabular}&\begin{tabular}{@{}c@{}} $2.41$ \\ $2.34$ \end{tabular}&\begin{tabular}{@{}c@{}} $5.1$ \\ $4.8$ \end{tabular}&\begin{tabular}{@{}c@{}} $2.25$ \\ $2.09$ \end{tabular}&\begin{tabular}{@{}c@{}} $5.1$ \\ $4.9$ \end{tabular}&\begin{tabular}{@{}c@{}} $3.07$ \\ $2.28$ \end{tabular}\\
\hline
32 & $5.0\cdot 10^{6}$ & \begin{tabular}{@{}c@{}} $10^{-4}$ \\ $10^{-1}$ \end{tabular}&\begin{tabular}{@{}c@{}} $7.4$ \\ $7.4$ \end{tabular}&\begin{tabular}{@{}c@{}} $2.41$ \\ $2.37$ \end{tabular}&\begin{tabular}{@{}c@{}} $7.5$ \\ $7.4$ \end{tabular}&\begin{tabular}{@{}c@{}} $2.27$ \\ $2.24$ \end{tabular}&\begin{tabular}{@{}c@{}} $7.5$ \\ $7.4$ \end{tabular}&\begin{tabular}{@{}c@{}} $3.31$ \\ $2.72$ \end{tabular}\\
\hline
128 & $7.7\cdot 10^{8}$ & \begin{tabular}{@{}c@{}} $10^{-4}$ \\ $10^{-1}$ \end{tabular}&\begin{tabular}{@{}c@{}} $14.4$ \\ $14.3$ \end{tabular}&\begi

In [235]:
s = ""

for d in d_list:
    s += str(d) + " & "
    kappa = mu_list[d]
    l = int(np.log10(kappa))
    s += "${:.1f}\\cdot 10^{{{}}}$".format(kappa/10**l, l) + " & "

    cur_list = ["$10^{{{}}}$".format(i) for i in res[d]["delta"]]
    s+= "\\begin{tabular}{@{}c@{}} " + " \\\\ ".join(cur_list) + " \\end{tabular}&"

    cur_list = ["${}$".format(i) for i in res[d]["iters_exact"]]
    s+= "\\begin{tabular}{@{}c@{}} " + " \\\\ ".join(cur_list) + " \\end{tabular}&"
    cur_list = ["${}$".format(i) for i in res[d]["exact,x0-x*"]]
    s+= "\\begin{tabular}{@{}c@{}} " + " \\\\ ".join(cur_list) + " \\end{tabular}&"
    cur_list = ["${}$".format(i) for i in res[d]["normg_exact"]]
    s+= "\\begin{tabular}{@{}c@{}} " + " \\\\ ".join(cur_list) + " \\end{tabular}"

    s+= "\\\\\n\\hline\n"
print(s)

8 & $2.1\cdot 10^{5}$ & \begin{tabular}{@{}c@{}} $10^{-4}$ \\ $10^{-1}$ \end{tabular}&\begin{tabular}{@{}c@{}} $14434$ \\ $1479$ \end{tabular}&\begin{tabular}{@{}c@{}} $5.1$ \\ $4.8$ \end{tabular}&\begin{tabular}{@{}c@{}} $2.41$ \\ $2.34$ \end{tabular}\\
\hline
32 & $5.0\cdot 10^{6}$ & \begin{tabular}{@{}c@{}} $10^{-4}$ \\ $10^{-1}$ \end{tabular}&\begin{tabular}{@{}c@{}} $59643$ \\ $12536$ \end{tabular}&\begin{tabular}{@{}c@{}} $7.4$ \\ $7.4$ \end{tabular}&\begin{tabular}{@{}c@{}} $2.41$ \\ $2.37$ \end{tabular}\\
\hline
128 & $7.7\cdot 10^{8}$ & \begin{tabular}{@{}c@{}} $10^{-4}$ \\ $10^{-1}$ \end{tabular}&\begin{tabular}{@{}c@{}} $921805$ \\ $264361$ \end{tabular}&\begin{tabular}{@{}c@{}} $14.4$ \\ $14.3$ \end{tabular}&\begin{tabular}{@{}c@{}} $2.43$ \\ $2.43$ \end{tabular}\\
\hline

